In [1]:
import pandas as pd
from reportlab.lib.pagesizes import letter
from reportlab.platypus import SimpleDocTemplate, Paragraph, Table, TableStyle, Spacer
from reportlab.lib import colors
from reportlab.lib.styles import getSampleStyleSheet
from datetime import datetime
import re  # Import the 're' module
from reportlab.lib.styles import getSampleStyleSheet, ParagraphStyle

In [2]:
# Read the datafiles using Pandas
path = r'c:\Users\dataanalyst\Desktop\Python projects\Reconciliation\datasets'
abc_data_file = pd.read_excel(path + '\Transactions2.xlsx', usecols = ['date_time', 'trn_ref', 'Amount', 'ACC/NO'], skiprows = 0)

# Renaming data columns
new_column_names = {
    'date_time': 'Date',
    'trn_ref': 'Reference',
    'ACC/NO': 'Account'
}
abc_data_file.rename(columns = new_column_names, inplace = True)

In [3]:
# Converting 'Tran Date' column to datetime using specific format
abc_data_file['Date'] = pd.to_datetime(abc_data_file['Date'], format='%d%m%Y', errors ='coerce')

# Create a copy of the ABC dataset for reference
abc_ref_data = abc_data_file.copy()

abc_ref_data['Date'] = abc_ref_data['Date'].dt.strftime('%d%m%Y').str.replace(r'[-/\\.]', '')

# Remove spaces, special characters, and replace spaces with zeros
def clean_column_value(value):
    cleaned_value = re.sub(r'[^0-9a-zA-Z]', '', str(value))
    if cleaned_value == '':
        return '0'
    return cleaned_value

# Apply the cleaning function to applicable columns (excluding 'Date')
for column in abc_data_file.columns:
    if column != 'Date':
        abc_ref_data[column] = abc_ref_data[column].apply(clean_column_value)

# Replace '-' in 'Amount' column with '0'
abc_ref_data['Amount'] = abc_ref_data['Amount'].str.replace('-', '0')

# Convert all columns to strings
for column in abc_ref_data.columns:
    abc_ref_data[column] = abc_ref_data[column].astype(str)

# Create a concatenated column for the ABC dataset
abc_ref_data['Concatenated_abc'] = (
    abc_ref_data['Date'].astype(str) +
    abc_ref_data['Reference'].astype(str) +
    abc_ref_data['Amount'].astype(str) +
    abc_ref_data['Account'].astype(str))


In [4]:
# Create a copy of abc_data_file and name it uploaded_dataset
uploaded_dataset = abc_ref_data.copy()
uploaded_dataset.drop(columns = 'Concatenated_abc', inplace = True)

uploaded_dataset['Concatenated_up'] = (
    uploaded_dataset['Date'].astype(str) +
    uploaded_dataset['Reference'].astype(str) +
    uploaded_dataset['Amount'].astype(str) +
    uploaded_dataset['Account'].astype(str))

uploaded_dataset = uploaded_dataset.iloc[:10]

In [5]:
# Merge and analyze data
merged_df = abc_ref_data.merge(uploaded_dataset, left_on='Concatenated_abc', right_on='Concatenated_up', how='outer', suffixes=('_abc', '_uploaded'))

# Create a new column 'Reconciliation Status' based on the match
reconciliation_column = 'Reconciliation Status'
merged_df[reconciliation_column] = 'Unreconciled'
merged_df.loc[merged_df['Concatenated_up'].notna(), reconciliation_column] = 'Reconciled'

# Separate reconciled and unreconciled rows for both datasets
reconciled_rows_abc = merged_df[merged_df[reconciliation_column] == 'Reconciled'][['Date_abc', 'Reference_abc', 'Amount_abc', 'Account_abc']]
reconciled_rows_uploaded = merged_df[merged_df[reconciliation_column] == 'Reconciled'][['Date_uploaded', 'Reference_uploaded', 'Amount_uploaded', 'Account_uploaded']]
unreconciled_rows_abc = merged_df[merged_df[reconciliation_column] == 'Unreconciled'][['Date_abc', 'Reference_abc', 'Amount_abc', 'Account_abc']]
#unreconciled_rows_uploaded = merged_df[merged_df[reconciliation_column] == 'Unreconciled'][['Date_uploaded', 'Reference_uploaded', 'Amount_uploaded', 'Account_uploaded']]

# Merge and analyze data with uploaded_dataset as the main dataset
merged_df_uploaded = pd.merge(uploaded_dataset, abc_ref_data, left_on='Concatenated_up', right_on='Concatenated_abc', how='left', suffixes=('_uploaded', '_abc'))

# Create a new column 'Reconciliation Status' based on the match
reconciliation_column_uploaded = 'Reconciliation Status'
merged_df_uploaded[reconciliation_column_uploaded] = 'Unreconciled'
merged_df_uploaded.loc[merged_df_uploaded['Concatenated_abc'].notna(), reconciliation_column_uploaded] = 'Reconciled'

# Separate unreconciled rows for the uploaded dataset
unreconciled_rows_uploaded = merged_df_uploaded[merged_df_uploaded[reconciliation_column_uploaded] == 'Unreconciled'][
    ['Date_uploaded', 'Reference_uploaded', 'Amount_uploaded', 'Account_uploaded', 'Concatenated_up']
]

# Print the results

# Print the results
print("Reconciled Rows (ABC):")
print(reconciled_rows_abc)

print("\nReconciled Rows (Uploaded Dataset):")
print(reconciled_rows_uploaded)

print("\nUnreconciled Rows (ABC):")
print(unreconciled_rows_abc)

print("\nUnreconciled Rows (Uploaded Dataset):")
print(unreconciled_rows_uploaded)


Reconciled Rows (ABC):
   Date_abc Reference_abc Amount_abc   Account_abc
0  31072023     147047986     500000    1200145399
1  31072023     147047975    1000000    1600086718
2  31072023     147047973     200000    6521400004
3  31072023   73121547909      10000  302222000000
4  31072023   73121158160     163500    3203274825
5  31072023   73121383362     200000    3203601231
6  31072023   73121730886      84000  305212000000
7  31072023   73121644749     100000       4120309
8  31072023   73121120936      10000  302222000000
9  31072023   73121812715      10000  302225000000

Reconciled Rows (Uploaded Dataset):
  Date_uploaded Reference_uploaded Amount_uploaded Account_uploaded
0      31072023          147047986          500000       1200145399
1      31072023          147047975         1000000       1600086718
2      31072023          147047973          200000       6521400004
3      31072023        73121547909           10000     302222000000
4      31072023        73121158160     

In [6]:
# Create the PDF document
doc = SimpleDocTemplate("Reconciliation File.pdf", pagesize = letter, leftMargin = 40, rightMargin = 40, topMargin = 40, bottomMargin = 40)
elements = []

# Define heading style and add heading to elements
heading_style = getSampleStyleSheet()['Heading1']
heading_style.fontName = 'Helvetica'
heading_style.fontSize = 18
heading_style.leading = 24
heading_text = "Reconciliation Summary Report"
heading = Paragraph(heading_text, style=heading_style)
elements.append(heading)

# Add spacer
elements.append(Spacer(1, 15))

# Define common style
common_style = getSampleStyleSheet()['Normal']
common_style.fontName = 'Helvetica'
common_style.leading = 16

# Calculate and add summary information for ABC Dataset
total_rows = len(abc_ref_data)  # Replace with actual ABC dataset row count
total_reconciled_rows = len(merged_df[merged_df['Reconciliation Status'] == 'Reconciled'])
total_unreconciled_rows = len(merged_df[merged_df['Reconciliation Status'] == 'Not Reconciled'])

current_datetime = datetime.now().strftime('%Y-%m-%d %H:%M:%S')
abc_dataset_summary_text = (
    f"<b>ABC Dataset Summary</b><br/>"
    f"Total Rows: {total_rows}<br/>"
    f"Reconciled Rows: {total_reconciled_rows}<br/>"
    f"Unreconciled Rows: {total_unreconciled_rows}<br/>"
    f"Recon Date and Time: {current_datetime}")

abc_dataset_summary_paragraph = Paragraph(abc_dataset_summary_text, common_style)
elements.append(abc_dataset_summary_paragraph)
elements.append(Spacer(1, 15))

# Calculate and add summary information for Uploaded Dataset
total_uploaded_rows = len(uploaded_dataset)  # Replace with the actual uploaded dataset row count
total_uploaded_reconciled_rows = len(merged_df[merged_df['Reconciliation Status'] == 'Reconciled'])
total_uploaded_unreconciled_rows = len(merged_df[merged_df['Reconciliation Status'] == 'Not Reconciled'])

uploaded_dataset_summary_text = (
    f"<b>Uploaded Dataset Summary</b><br/>"
    f"Total Rows: {total_uploaded_rows}<br/>"
    f"Reconciled Rows: {total_uploaded_reconciled_rows}<br/>"
    f"Unreconciled Rows: {total_uploaded_unreconciled_rows}")

uploaded_dataset_summary_paragraph = Paragraph(uploaded_dataset_summary_text, common_style)
uploaded_dataset_summary_paragraph.alignment = 2  # Set alignment to the right
elements.append(uploaded_dataset_summary_paragraph)
elements.append(Spacer(1, 15))

# Assuming 'merged_df' is a DataFrame containing your data
merged_df['Amount_abc'] = pd.to_numeric(merged_df['Amount_abc'], errors='coerce')

# Display reconciled and unreconciled rows
reconciled_data = []
unreconciled_data = []

for _, row in merged_df.iterrows():
    status = row['Reconciliation Status']
    
    # Check if the 'Reconciliation Reason' column exists in the row
    if 'Reconciliation Reason' in row:
        reason = row['Reconciliation Reason']
    else:
        reason = ""  # If the column doesn't exist, set the reason to an empty string
        
    # Format the Amount value as an integer (remove decimals)
    formatted_amount = "{:,.0f}".format(row['Amount_abc'])  # Format as integer

    row_data = [
        row['Reference_abc'],
        formatted_amount,
        row['Account_abc'],
        status,
    ]

    if status == 'Reconciled':
        reconciled_data.append(row_data)
    else:
        unreconciled_data.append(row_data + [reason])

# Create a TableStyle for the table
table_style = TableStyle([
    ('BACKGROUND', (0, 0), (-1, 0), colors.gray),
    ('ALIGN', (0, 0), (-1, -1), 'CENTER'),
    ('FONTNAME', (0, 0), (-1, 0), 'Helvetica-Bold'),
    ('BOTTOMPADDING', (0, 0), (-1, 0), 10),
    ('LEFTPADDING', (0, 0), (-1, 0), 10),
    ('RIGHTPADDING', (0, 0), (-1, 0), 10),
    ('BACKGROUND', (0, 0), (-1, -1), colors.beige),
    ('GRID', (0, 0), (-1, -1), 1, colors.black),
    ('BOX', (0, 0), (-1, -1), 1, colors.black),
])

# Create a function to generate and format the table with status colors
def create_formatted_table(data, title, show_reason_column=True):
    num_columns = 6 if not show_reason_column else 7
    table_data = [
        [Paragraph("<b>Reference</b>", common_style),
         Paragraph("<b>Amount</b>", common_style),
         Paragraph("<b>Account</b>", common_style),
         "Status",
         "Reason"]
    ] + data
    
    col_widths = [80, 100, 80, 120, 60, 150, 120] if not show_reason_column else [80, 100, 80, 120, 60, 150, 150]
    table = Table(table_data, colWidths=col_widths[:num_columns])
    
    # Set alignment to the right for all columns except the last two
    table.setStyle(TableStyle([
        ('ALIGN', (0, 0), (-1, 0), 'CENTER'),  # Align header cells centrally
        ('ALIGN', (0, 0), (-3, -1), 'RIGHT'),  # Align Reference and Amount columns to the right
        ('ALIGN', (-2, 0), (-1, -1), 'CENTER'),  # Align Status and Reason columns to the center
        ('FONTNAME', (0, 0), (-1, 0), 'Helvetica-Bold'),
        ('BOTTOMPADDING', (0, 0), (-1, 0), 10),
        ('LEFTPADDING', (0, 0), (-1, 0), 10),
        ('RIGHTPADDING', (0, 0), (-1, 0), 10),
        ('BACKGROUND', (0, 0), (-1, -1), colors.beige),
        ('GRID', (0, 0), (-1, -1), 1, colors.black),
        ('BOX', (0, 0), (-1, -1), 1, colors.black),
        
    ]))
    
    title_style = getSampleStyleSheet()['Heading2']
    title_style.alignment = 1
    title_element = Paragraph(title, title_style)
    elements.append(title_element)
    elements.append(table)
    
    for i, row in enumerate(table_data[1:], start=0):
        status = row[4] if len(row) > 4 else ""
        
        if status == 'Reconciled':
            table.setStyle(TableStyle([
                ('BACKGROUND', (0, i+1), (-1, i+1), colors.lightgreen),
            ]))
        elif status == 'Unreconciled':
            table.setStyle(TableStyle([
                ('BACKGROUND', (0, i+1), (-1, i+1), colors.lightcoral),
            ]))

# Add the formatted reconciled table without the "Reason" column
create_formatted_table(reconciled_data, "Reconciled Data", show_reason_column=False)

elements.append(Spacer(1, 15))

# Add the formatted unreconciled table with the "Reason" column
create_formatted_table(unreconciled_data, "Unreconciled Data", show_reason_column=True)

# Print PDF generation messages
print("Generating PDF...")

# Build the PDF document
doc.build(elements)

# Print PDF generation completion message
print("PDF generation complete.")

Generating PDF...
PDF generation complete.
